<a href="https://colab.research.google.com/github/abay-qkt/google-timeline-visualizer/blob/main/google_timeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/location-history.json"

# 準備

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import ipywidgets as widgets

from google.colab import output
output.enable_custom_widget_manager()

## タイムラインデータのロード

In [ ]:
df = pd.read_json(path)
df["tz_info"]=df["startTime"].str[23:]
df["endTime"] = pd.to_datetime(df["endTime"].str[:23])
df["startTime"] = pd.to_datetime(df["startTime"].str[:23])
df.set_index(["endTime","startTime","tz_info"],inplace=True)

# timelinePath
tp = df["timelinePath"].dropna()
tp = pd.json_normalize(tp.explode()).set_index(tp.explode().index)
tp[["path_lat","path_lon"]] = tp["point"].str[4:].str.split(",",expand=True).astype(float)
tp = tp.drop(["point"],axis=1).reset_index()
tp["durationMinutesOffsetFromStartTime"]=tp["durationMinutesOffsetFromStartTime"].astype(int)
tp["pointTime"]=(
    tp["startTime"]  #  startTime+durationMinutesOffsetFromStartTime時点でその緯度経度に居たと解釈する
    +pd.to_timedelta(tp["durationMinutesOffsetFromStartTime"],unit='minute')
    +pd.to_timedelta(9,unit='hour')  # 日本時間に変換する
)
tp["prevTime"]=tp["pointTime"].shift()  # 直前の時刻を示す列も用意しておく

# timelineMemory
tm = df["timelineMemory"].dropna()
tm = pd.json_normalize(tm).set_index(tm.index).explode("destinations")
tm["destinations"]=tm["destinations"].map(lambda x: x["identifier"])
tm["distanceFromOriginKms"]=tm["distanceFromOriginKms"].astype(int)
tm.reset_index(inplace=True)

# activity
ac = df["activity"].dropna()
ac = pd.json_normalize(ac).set_index(ac.index)
ac[["start_lat","start_lon"]]=ac["start"].str[4:].str.split(",",expand=True).astype(float)
ac[["end_lat","end_lon"]]=ac["end"].str[4:].str.split(",",expand=True).astype(float)
ac.drop(["start","end"],axis=1,inplace=True)
ac["distanceMeters"]=ac["distanceMeters"].astype(float)
ac.reset_index(inplace=True)

# visit
vt = df["visit"].dropna()
vt = pd.json_normalize(vt).set_index(vt.index)
vt[["place_lat","place_lon"]]=vt["topCandidate.placeLocation"].str[4:].str.split(",",expand=True).astype(float)
vt.drop(["topCandidate.placeLocation"],axis=1,inplace=True)
vt["hierarchyLevel"]=vt["hierarchyLevel"].astype(int)
vt["topCandidate.probability"]=vt["topCandidate.probability"].astype(float)
vt.reset_index(inplace=True)

In [ ]:
# 訪問地ごとに集計
vt_info = vt.copy()
vt_info["startDate"]=vt_info["startTime"].dt.date.astype(str)
vt_info = (
    vt_info
    .groupby(["topCandidate.placeID","place_lat","place_lon"])["startDate"]
    .agg(["nunique","max"])
    .sort_values("max")
    .rename(columns={"nunique":"visit_count","max":"last_visit"})
    .reset_index()
)

## 訪問地プロット関数

In [ ]:
def plot_visiting(vt):
  fig = px.scatter_mapbox(vt_info,lat='place_lat',lon='place_lon',hover_data=["visit_count","last_visit"],mapbox_style ='carto-positron')
  fig.update_traces(marker=dict(color='red',size=10))
  fig.update_layout(height=800,width=1200)
  fig.show(config={"scrollZoom":True})

## 経路プロット関数

In [ ]:
# Date Pickerウィジェットを作成
date_picker = widgets.DatePicker(
    description='Select Date',
    disabled=False
)
map_output = widgets.Output()  # 描画のためのOutputウィジェットを作成
def plot_path(tp,vt):
  fig1 = px.line_mapbox(tp.reset_index(),lat='path_lat',lon='path_lon',hover_data=["startTime","endTime"],mapbox_style ='carto-positron')
  fig2 = px.scatter_mapbox(vt.reset_index(),lat='place_lat',lon='place_lon',hover_data=["startTime","endTime"],mapbox_style ='carto-positron')
  fig2.update_traces(marker=dict(color='red',size=10))
  fig1.add_traces(fig2.data)
  fig1.update_layout(height=600,width=1000)
  fig1.show(config={"scrollZoom":True})

# コールバック関数を定義
def on_date_change(change):
    selected_date = change['new']
    if selected_date is not None:
        tp_=tp[tp["startTime"].dt.date==selected_date]
        vt_=vt[vt["startTime"].dt.date==selected_date]
        # グラフを描画
        if not vt_.empty:
            with map_output:
                map_output.clear_output()
                plot_path(tp_,vt_)
        else:
            with map_output:
                map_output.clear_output()
                print("No data available for the selected date.")

date_picker.observe(on_date_change, names='value')  # ウィジェットにコールバック関数をバインド
plot_path_ui = widgets.VBox([date_picker,map_output])

## 訪問地kml保存

In [ ]:
!pip install simplekml

In [ ]:
import simplekml

def save_visit(vt_info):
  kml = simplekml.Kml()

  style = simplekml.Style()
  style.iconstyle.icon.href = 'https://maps.google.com/mapfiles/ms/micons/red-dot.png'
  style.iconstyle.scale = 3
  style.labelstyle.scale = 0  # ラベルのスケールを0に設定（つまり非表示になる）

  for i in range(len(vt_info)):
      x = vt_info.iloc[i]
      pt = kml.newpoint(name=x["topCandidate.placeID"],
                        coords=[( x["place_lon"],x["place_lat"])],
                        description=f'訪問日数:{x["visit_count"]} 最終訪問日:{x["last_visit"]}')
      pt.timestamp.when = x["last_visit"]
      pt.style = style
  kml.save(path = "visit.kml")

In [ ]:
save_visit(vt_info)

# 可視化

## 訪問地

In [ ]:
plot_visiting(vt)

## 経路（日別）

In [ ]:
display(plot_path_ui)